In [1]:
from dotenv import load_dotenv
import instructor
from openai import OpenAI
import os
from pathlib import Path
from pydantic import BaseModel, Field, StringConstraints, conlist, constr, field_validator
from typing_extensions import Annotated


In [4]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# main constants

GPT_MODEL = "gpt-4o" # points to latest GPT model

#instantiate client
client = instructor.from_openai(OpenAI(), mode=instructor.Mode.TOOLS)

In [5]:
# simple goal

class SimpleGoal(BaseModel):
    """Given a context, extract and generate a concise and straightforward main goal based on a specified action and target."""
    
    action: str = Field(..., 
                        description="The decisive action to be executed, using strong and directive verbs")
    target: str = Field(..., 
                        description="The specific item or resource on which the action is to be performed, described in simple language.")
    goal: Annotated[str, StringConstraints(max_length=50)] = Field(..., 
                                                                   description="A concise, imperative sentence that combines the action and the target into a clear, understandable main goal.")

In [6]:
# list of simple goals

class SimpleGoals(BaseModel):
    """Given a context, extract and infer a list of goals"""

    goals: list[SimpleGoal]

In [7]:
# n length list of simple goals 

class SimpleGoals(BaseModel):
    """Given a context, extract and infer a {min_length} list of goals"""

    goals: conlist(SimpleGoal, min_length=3) # type: ignore

    

In [18]:
# product or business goal

class ComplexGoal(BaseModel):
    """Given a context, extract and infer a specific, main product or business goal"""
    
    verb: str = Field(..., description="A clear, singular action.")
    quantitative_metric: str = Field(..., description="The numerical part of the goal, including either a number or a percentage.",
                                      examples=["20%", "30 minutes", "90%", "5 additional metropolitan areas", "3-5"])
    qualitative_metric: str = Field(..., description="A concise description of what the goal targets, without the numeric component.",
                                     examples=["increase the number of bookings", "reduce average wait time for pickups",
                                               "achieve a customer satisfaction rate", "expand service",
                                               "increase recycling rate of collected rubbish"])
    focus: str = Field(..., description="A concise, well-defined area of impact.")
    timeframe: str = Field(None, description="A specific, achievable deadline.")
    goal: str = Field(..., description="Craft a succinct, declarative sentence that integrates the verb, quantitative and qualitative metrics, focus, and timeframe, ensuring to include a specific number or percentage.")

In [ ]:
         "verb": "decrease",
                    "quantitative_metric": "10%",
                    "qualitative_metric": "reduce churn rate",
                    "focus": "subscription services",
                    "timeframe": "by the end of Q4",
                    "goal": "Decrease churn rate by 10% in subscription services by the end of Q4."
                },
                {
                    "verb": "raise",
                    "quantitative_metric": "$1 million",
                    "qualitative_metric": "raise capital# revised product or business goal with Anotated constraints ???

from pydantic import BaseModel, Field
from pydantic.config import ConfigDict

class ComplexGoal(BaseModel):
    """Given a context, extract and infer a specific, main product or business goal"""
    
    verb: str = Field(..., description="A clear, singular action.")
    quantitative_metric: str = Field(..., description="The numerical part of the goal, including either a number or a percentage.",
                                      examples=["20%", "30 minutes", "90%", "5 additional metropolitan areas", "3-5"])
    qualitative_metric: str = Field(..., description="A concise description of what the goal targets, without the numeric component.",
                                     examples=["increase the number of bookings", "reduce average wait time for pickups",
                                               "achieve a customer satisfaction rate", "expand service",
                                               "increase recycling rate of collected rubbish"])
    focus: str = Field(..., description="A concise, well-defined area of impact.")
    timeframe: str = Field(None, description="A specific, achievable deadline.")
    goal: str = Field(..., description="Craft a succinct, declarative sentence that integrates the verb, quantitative and qualitative metrics, focus, and timeframe, ensuring to include a specific number or percentage.")

    model_config = ConfigDict(
        json_schema_extra={
            "examples": [
                {
                    "verb": "boost",
                    "quantitative_metric": "15%",
                    "qualitative_metric": "enhance user engagement",
                    "focus": "mobile app usage",
                    "timeframe": "in the next quarter",
                    "goal": "Boost user engagement by 15% in mobile app usage in the next quarter."
                },
                {
           ",
                    "focus": "investment funding",
                    "timeframe": "within 12 months",
                    "goal": "Raise $1 million in investment funding within 12 months."
                },
                {
                    "verb": "improve",
                    "quantitative_metric": "50%",
                    "qualitative_metric": "improve response rate",
                    "focus": "customer support",
                    "timeframe": "by mid-year",
                    "goal": "Improve response rate by 50% in customer support by mid-year."
                },
                {
                    "verb": "expand",
                    "quantitative_metric": "10 new cities",
                    "qualitative_metric": "expand market presence",
                    "focus": "urban areas",
                    "timeframe": "over the next 18 months",
                    "goal": "Expand market presence to 10 new cities in urban areas over the next 18 months."
                }
            ]
        }
    )


In [19]:
class ComplexGoals(BaseModel):
    """Given a context, extract and infer a list of goals"""

    goals: list[ComplexGoal]

In [26]:
class ComplexGoals(BaseModel):
    """Given a context, extract and infer a {min_length} list of goals"""

    goals: conlist(ComplexGoal, min_length=5) # type: ignore

In [28]:
# wrapper

def wrapper(system_prompt: str | None = None, user_context: str | list = None, response_model: BaseModel | None = None, max_retries: int = 3):
    """Wrapper function to generate LLM completion"""
    messages = []
    if system_prompt is not None:
        messages.append({"role": "system", "content": system_prompt})
    if user_context is not None:
        messages.append({"role": "user", "content": user_context})

    completion = client.chat.completions.create(
        model=GPT_MODEL,
        response_model=response_model,
        max_retries=max_retries,
        messages=messages
    )
    return completion



In [17]:
context = """
I have lots of youtube podcasts I want to watch. But I don't have the time to do so. 
Would be great if I can just put in a url or video id and get back a summary with all the main points.
Saves me so much time.
"""

In [35]:
context = """
!!! tip "Hypothesis"

    A dedicated platform team accelerates AI value creation and capture for B2B SaaS companies

Innovations are adopted quickly if they are easy to make, easy to use, fit existing patterns, and have visible results. Generative AI has abruptly changed the innovation landscape by acclerating the utility and availability of AI products and services.

While the vast majority of consumers and companies do not make technology, they are actively searching for value in this space. Consequently, technology companies are trying to implement AI features that create value for their customers as fast as possible.

The long tail of B2B SaaS companies have a unique set of challenges. They have enterprise client traction and skilled development teams but lack extra capital and in-house AI/ML resources, making it hard to bootstrap production grade AI capabilities and features.

A dedicated platform team helps B2B SaaS companies stay competitive by quickly matching competitors advances and meeting client expectations for new experiences.

This team drives innovation, positioning the company as a leader in AI, and reduces risk by providing systemic infrastructure and resources to prevent project failures.

It specifically helps:

1. Product teams launch AI features
2. Business teams automate workflows
3. Leaders unify AI strategy, standards and voice
4. Clients adopt third-party AI products
5. Staff upskill AI knowledge and skills

Ultimately, platform teams shape organisational culture, ensuring that responsibility and enthusiasm for AI are shared by everyone, not just a small group of evangelists.

Good platform teams function as skunkworks, rapid execution engines, and guardrails, ensuring the company quickly and safely integrates AI across all operations in order to benefit clients and establish AI leadership amongst competitors.

## Ideas

When bootstrapping platform teams, only speed and outcomes matter. Fast, cheap value creation determines whether platforms continue or are killed off because resources are limited, track records nonexistent and executive patience is thin.

Additionally, platform teams must act for all functions and departments. Specifically, they should tackle cross disciplinary needs - at least product, design and engineering to start with - and champion fairness - equitable platform access to all products - as foundational principles for success.

In this environment, prioritisation is king and selecting the correct set of valuable and feasible initiatives to iterate on and build company confidence is crucial.

The list below sequences and categorises a core set of ideas for new platform teams to tackle.
"""

In [36]:
response = wrapper(user_context=context, response_model=SimpleGoal)
response

SimpleGoal(action='Establish', target='dedicated platform team for B2B SaaS companies', goal='Establish a dedicated platform team')

In [37]:
response = wrapper(user_context=context, response_model=SimpleGoals)
print(f"No. of goals is : {len(response.goals)}")
response.goals

No. of goals is : 5


[SimpleGoal(action='Launch', target='AI features', goal='Launch AI features quickly'),
 SimpleGoal(action='Automate', target='workflows', goal='Automate workflows efficiently'),
 SimpleGoal(action='Unify', target='AI strategy', goal='Unify AI strategy effectively'),
 SimpleGoal(action='Adopt', target='third-party AI products', goal='Adopt third-party AI products'),
 SimpleGoal(action='Upskill', target='staff', goal='Upskill staff in AI')]

In [38]:
response = wrapper(user_context=context, response_model=ComplexGoal)
response.goal

'Launch 5 AI features for the product teams of B2B SaaS companies within 6 months.'

In [39]:
response = wrapper(user_context=context, response_model=ComplexGoals)
for goal in response.goals:
    print(goal.goal)

Launch 5 new AI features for product teams within the next 6 months.
Automate 50% of business workflows for business teams within 1 year.
Unify 100% of AI strategy, standards, and voice for leadership within 2 years.
Facilitate 30% adoption of third-party AI products by clients within 1 year.
Improve 75% of staff AI knowledge and skills within 18 months.
